In [92]:
from mido import MidiFile, MidiTrack, Message
import numpy as np
import os 
import pretty_midi

def midi_to_array(midi_path):
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(midi_path)
    
    # Get the piano roll representation
    piano_roll = midi_data.get_piano_roll(fs=1)  # Set fs to control time resolution
    
    # Transpose the piano roll to have time steps as rows and pitches as columns
    piano_roll = piano_roll.T
    
    return piano_roll

# def midi_to_notes(midi_file, length = 1000):
#     mid = MidiFile(midi_file)
#     notes = []
#     for msg in mid:
#         if not msg.is_meta and msg.channel == 0 and msg.type == "note_on":
#             data = msg.bytes()
#             if data[2] != 0:
#                 notes.append(data[1])
#     notes = np.array(notes)
#     notes = notes[:length]
#     return notes

# Example usage
midi_path = 'maestro-v3.0.0-midi/maestro-v3.0.0/2018/MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1.midi'
midi_pathg = "maestro-v3.0.0-midi/maestro-v3.0.0/2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi"
midi_array = midi_to_array(midi_path)
midi_arrayg = midi_to_array(midi_pathg)
print(midi_array.shape)  # Print the shape of the resulting numpy array
print(midi_arrayg.shape)

(582, 128)
(703, 128)


In [84]:
data = []
for file in os.listdir("maestro-v3.0.0-midi/maestro-v3.0.0/2018"):
    if file.endswith(".midi"):
        data.append(midi_to_array("maestro-v3.0.0-midi/maestro-v3.0.0/2018/" + file))

In [85]:
#pad the data
max_len = max([x.shape[0] for x in data])
max_len

2562

In [86]:
padded_data = []
for x in data:
    pad = np.zeros((max_len - x.shape[0], x.shape[1]))
    padded_data.append(np.concatenate((x, pad), axis=0))


In [87]:
padded_data[0].shape

(2562, 128)

In [93]:
#conver padded data to midi files
padded_data[0] = padded_data[0].T
padded_data[0].shape

#convert to midi
def array_to_midi(array, name):
    array = array.T
    midi = MidiFile()
    track = MidiTrack()
    midi.tracks.append(track)
    for i in range(array.shape[0]):
        if array[i, 0] != 0:
            track.append(Message('note_on', note=int(array[i, 0]), velocity=int(array[i, 1]), time=int(array[i, 2])))
    midi.save(name)

array_to_midi(data[0], "test.midi")


In [29]:
from torch.utils.data import Dataset, DataLoader
import torch

class MidiDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def collate_fn(self, batch):
        batch = torch.tensor(batch)
        batch = batch.transpose(0, 1)
        return batch

dataset = MidiDataset(padded_data, max_len)


In [36]:
class Generator(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.lstm = torch.nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, output_size)
        self.softmax = torch.nn.Softmax(dim=2)

    def forward(self, x, hidden):
        x, hidden = self.lstm(x, hidden)
        x = self.linear(x)
        x = self.softmax(x)
        return x, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(1000, batch_size, self.hidden_size), torch.zeros(1000, batch_size, self.hidden_size))

In [37]:
class Disciminator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = torch.nn.LSTM(1, 64, 2, batch_first=True)
        self.linear = torch.nn.Linear(64, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

In [38]:
class GAN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GAN, self).__init__()
        self.generator = Generator(input_size, hidden_size, output_size)
        self.discriminator = Disciminator()

    def forward(self, x, hidden):
        x, hidden = self.generator(x, hidden)
        x = self.discriminator(x)
        return x, hidden

    def init_hidden(self, batch_size):
        return self.generator.init_hidden(batch_size)

def train(dataloader, model, optimizer, criterion, epochs):
    for epoch in range(epochs):
        for i, batch in enumerate(dataloader):
            batch_size = batch.shape[1]
            hidden = model.init_hidden(batch_size)
            for j in range(batch.shape[0]):
                optimizer.zero_grad()
                x = batch[j].unsqueeze(0).unsqueeze(2).float()
                y = torch.ones(batch_size, 1)
                y_hat, hidden = model(x, hidden)
                loss = criterion(y_hat, y)
                loss.backward()
                optimizer.step()
                print(loss.item())

